# Activity Classifier

Goal
:   1. Predict the activity action of a sequence of events
    2. Predict the case IDs of sequences of events



In [2]:
import pandas as pd
from preprocessing.preprocessing import extract_features
import ast

In [3]:
df_train_in = pd.read_csv('../../data_v3/hr_extended_features.csv', converters={"MessageAttributes": ast.literal_eval})
df_test =pd.read_csv('../../data_v3/hr_emitted_events_test.csv')

In [6]:
df_train_in[["MessageAttributes","query"]]

,MessageAttributes,query
0,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",NaN
1,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",NaN
2,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",NaN
3,"{'pgsql.type': 'Simple query', 'pgsql.length':...","INSERT INTO ""res_users_log"" (""id"", ""create_uid..."
4,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",NaN
...,...,...
33068,"{'pgsql.type': 'Simple query', 'pgsql.length':...","UPDATE ""mail_message_res_partner_needaction_re..."
33069,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",NaN
33070,"{'': 'POST /xmlrpc/2/object HTTP/1.1\r\n', '_w...",NaN
33071,"{'pgsql.type': 'Simple query', 'pgsql.length':...","UPDATE ""hr_applicant"" SET ""active""=false,""writ..."


In [7]:
df_train_in[["MessageAttributes","query"]]

,MessageAttributes,query
0,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",NaN
1,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",NaN
2,"{'': 'POST /xmlrpc/2/common HTTP/1.1\r\n', '_w...",NaN
3,"{'pgsql.type': 'Simple query', 'pgsql.length':...","INSERT INTO ""res_users_log"" (""id"", ""create_uid..."
4,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",NaN
...,...,...
33068,"{'pgsql.type': 'Simple query', 'pgsql.length':...","UPDATE ""mail_message_res_partner_needaction_re..."
33069,"{'': 'HTTP/1.0 200 OK\r\n', '_ws.expert': 'Exp...",NaN
33070,"{'': 'POST /xmlrpc/2/object HTTP/1.1\r\n', '_w...",NaN
33071,"{'pgsql.type': 'Simple query', 'pgsql.length':...","UPDATE ""hr_applicant"" SET ""active""=false,""writ..."


## Preprocessing

In [5]:
# Mark groups of Instance Number and BusinessActivity with sequence numbers
df_train = df_train.sort_values(by=["InstanceNumber", "BusinessActivity", "frame.number"])
df_train["SequenceNumber"] = df_train.groupby(["BusinessActivity", "InstanceNumber"]).ngroup()
# Align Sequence Numbers so that they start at 0
df_train["SequenceNumber"] -= df_train['SequenceNumber'].min()

In [6]:
def sequence_by_activities(data, seq_data):
    return [data[seq_data == i] for i in range(seq_data.max())]

In [7]:
feature_cols = ["event_with_roles", "request_method_call", "selective_file_data", 
        "origin_method","origin_file_data"]


In [8]:
# List of dataframes each containing one activity sequence
train_activity_sequences = sequence_by_activities(df_train, df_train["SequenceNumber"])

test_activity_sequences = sequence_by_activities(df_test, df_test["stack_idx"])

In [9]:
# We train a crf with linked / not linked features on the sequential data and evaluate on the test data

In [10]:
# Sequences without window features

def dict_to_feature_sequence(dict):
    return [{**d, "bias": 1.0} for d in dict]

def df_to_features(df):
    return dict_to_feature_sequence(df.to_dict("records"))

train_features_seq = [df_to_features(df[feature_cols]) for df in train_activity_sequences]
train_labels_seq = [df["BusinessActivity"].values for df in train_activity_sequences]


test_features_seq = [df_to_features(df[feature_cols]) for df in test_activity_sequences]
test_labels_seq = [df["activity_name"].values for df in test_activity_sequences]

In [11]:
# Single Events no window features 

def dict_to_feature(dict):
    return [[{**d, "bias": 1.0}] for d in dict]

def extract_labels(labels):
    return [[y] for y in labels]

train_features = dict_to_feature(df_train[feature_cols].to_dict("records"))
train_labels = extract_labels(df_train["BusinessActivity"])

test_features = dict_to_feature(df_test[feature_cols].to_dict("records"))
test_labels = extract_labels(df_test["activity_name"])

In [12]:
# Single Events w. window features

# Apply sequencing - flatten later

def seq2features(seq, bw, fw): 
    return [event2features(seq, i, bw, fw) for i in range(len(seq))]

def event2features(seq, i, bw, fw):
    features = {"bias": 1.0}
    
    features.update({
        f"0:{k}": v for k,v in seq[i].items()
    })
    
    for j in range(1, bw+1): 
        index = i-j
        if index >= 0: 
            features.update({
                f"-{j}:{k}": v for k,v in seq[index].items()
            })
        else: 
            features.update({
                 f"-{j}:{k}": "NoMessage" for k,_ in seq[i].items()
            })
        
    for j in range(1,fw+1): 
        index = i + j
        if index < len(seq): 
             features.update({
                f"+{j}:{k}": v for k,v in seq[index].items()
            })
        else: 
            features.update({
                 f"+{j}:{k}": "NoMessage" for k,_ in seq[i].items()
            })
            
    return features

seq2features(train_activity_sequences[0][feature_cols].to_dict("records"), 5, 5)

[{'bias': 1.0,
  '0:event_with_roles': 'End Point (HR Manager)->Odoo Application:[HttpRequest:POST /xmlrpc/2/common HTTP/1.1\\r\\n]',
  '0:request_method_call': 'version',
  '0:selective_file_data': 'version',
  '0:origin_method': '',
  '0:origin_file_data': '',
  '-1:event_with_roles': 'NoMessage',
  '-1:request_method_call': 'NoMessage',
  '-1:selective_file_data': 'NoMessage',
  '-1:origin_method': 'NoMessage',
  '-1:origin_file_data': 'NoMessage',
  '-2:event_with_roles': 'NoMessage',
  '-2:request_method_call': 'NoMessage',
  '-2:selective_file_data': 'NoMessage',
  '-2:origin_method': 'NoMessage',
  '-2:origin_file_data': 'NoMessage',
  '-3:event_with_roles': 'NoMessage',
  '-3:request_method_call': 'NoMessage',
  '-3:selective_file_data': 'NoMessage',
  '-3:origin_method': 'NoMessage',
  '-3:origin_file_data': 'NoMessage',
  '-4:event_with_roles': 'NoMessage',
  '-4:request_method_call': 'NoMessage',
  '-4:selective_file_data': 'NoMessage',
  '-4:origin_method': 'NoMessage',
  '

In [13]:
train_features_seq_window = [seq2features(seq[feature_cols].to_dict("records"), 5,5) for seq in train_activity_sequences]
train_labels_seq_window = [seq["BusinessActivity"] for seq in train_activity_sequences]

In [14]:
test_features_seq_window = [seq2features(seq[feature_cols].to_dict("records"), 5,5) for seq in test_activity_sequences]
test_labels_seq_window = [seq["activity_name"].tolist() for seq in test_activity_sequences]

In [15]:
test_labels_seq_window

[['GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity',
  'GenerateJobApplicationActivity'],
 ['ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewActivity',
  'ResumeReviewAc

In [16]:
from sklearn.model_selection import train_test_split

APPLY_SEQUENCES = False
EVAL_ON_IL = False

if APPLY_SEQUENCES: 
    train_features = train_features_seq
    test_features = test_features_seq
    train_labels = train_labels_seq
    test_labels = test_labels_seq

    
else: 
    X_train, X_test, y_train, y_test = train_test_split(train_features, train_labels, test_size=0.3, random_state=42)

In [17]:
X_train = train_features_seq_window
X_test = test_features_seq_window
y_train = train_labels_seq_window
y_test = test_labels_seq_window

In [18]:
def flatten(list_of_list): 
    return [item for sublist in list_of_list for item in sublist]

def flatten_and_encapsulate(list_of_list):
    return [[item] for sublist in list_of_list for item in sublist]


X_train = flatten_and_encapsulate(train_features_seq_window)
X_test_seq = test_features_seq_window
X_test = flatten_and_encapsulate(test_features_seq_window)
y_train = flatten_and_encapsulate(train_labels_seq_window)
y_test_seq = test_labels_seq_window
y_test = flatten_and_encapsulate(test_labels_seq_window)

# Model Training

In [19]:
%%time
import sklearn_crfsuite

crf = sklearn_crfsuite.CRF(
    max_iterations=200,
    c1=0.1,
    c2=0.01,
    all_possible_transitions=True
    #all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 7 s, sys: 48.4 ms, total: 7.05 s
Wall time: 7.12 s


CRF(all_possible_transitions=True, c1=0.1, c2=0.01, max_iterations=200)

In [20]:
from sklearn.metrics import multilabel_confusion_matrix
from sklearn_crfsuite import metrics


def flatten(xss):
    return [x for xs in xss for x in xs]


def evaluate(model, x, y_true):
    y_pred = model.predict(x)
    print(metrics.flat_f1_score(y_true, y_pred, average='macro', labels=model.classes_))
    print(metrics.flat_classification_report(y_true, y_pred, model.classes_))
    [print(label, "\n", matrix) for matrix, label in
     zip(multilabel_confusion_matrix(flatten(y_true), flatten(y_pred), labels=model.classes_), model.classes_)]


In [21]:
y_pred = crf.predict(X_test)

In [22]:
evaluate(crf, X_test_seq, y_test_seq)

0.7298196595698789
                                 precision    recall  f1-score   support

               ContractProposal       1.00      0.46      0.63        63
 GenerateJobApplicationActivity       0.97      0.99      0.98       136
         PerformAnInterviewCall       0.89      0.83      0.86       134
      PerformAnInterviewMeeting       0.23      0.97      0.37        58
           ResumeReviewActivity       0.94      0.86      0.90       465
ScheduleAnInterviewActivityCall       0.82      0.72      0.77       228
     ScheduleAnInterviewMeeting       0.85      0.47      0.60       186

                       accuracy                           0.77      1270
                      macro avg       0.81      0.76      0.73      1270
                   weighted avg       0.87      0.77      0.80      1270

ContractProposal 
 [[1207    0]
 [  34   29]]
GenerateJobApplicationActivity 
 [[1130    4]
 [   1  135]]
PerformAnInterviewCall 
 [[1122   14]
 [  23  111]]
PerformAnIntervie

In [23]:
# Try Majority Voting

In [24]:
len(X_test_seq[0])

15

In [25]:
test_activity_sequences[0]

,Unnamed: 0,BusinessActivity,InstanceNumber,frame.number,synthetic_sniff_time,event_with_roles,request_method_call,file_data,selective_file_data,origin_method,origin_file_data,ActivityAction,stack_idx,activity_name,bp_id
0,0,HrRecruitmentProcess,over,17,2020-09-18 13:37:23.875912,End Point (HR Manager)->Odoo Application:[Http...,version,[],version,NaN,NaN,Activity Start,0,GenerateJobApplicationActivity,1224.0
1,1,HrRecruitmentProcess,over,49,2020-09-18 13:45:19.523472,Odoo Application->End Point (HR Manager):[Http...,server_version,"['server_version', '12.0-20190820', 'server_ve...",server_version,version,version,NoAction,0,GenerateJobApplicationActivity,1224.0
2,2,HrRecruitmentProcess,over,57,2020-09-18 13:45:48.836403,End Point (HR Manager)->Odoo Application:[Http...,authenticate,"['odoo01', 'dana.wireless@gmail.com', '1234567...",NaN,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
3,3,HrRecruitmentProcess,over,101,2020-09-18 14:30:06.242117,Odoo Application->db Server/Mail Server:[Pgsql...,NaN,NaN,NaN,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
4,4,HrRecruitmentProcess,over,110,2020-09-18 14:31:05.748067,Odoo Application->End Point (HR Manager):[Http...,IsNumber,['6'],IsNumber,authenticate,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
5,5,HrRecruitmentProcess,over,118,2020-09-18 14:31:27.680260,End Point (HR Manager)->Odoo Application:[Http...,execute_kw,"['odoo01', '6', '123456789', 'hr.job', 'search...",hr.job_search_read,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
6,6,HrRecruitmentProcess,over,175,2020-09-18 14:39:35.469034,Odoo Application->End Point (HR Manager):[Http...,id,"['id', '1', 'name', 'VP R&D', 'id', '2', 'name...",name,execute_kw,hr.job_search_read,NoAction,0,GenerateJobApplicationActivity,1224.0
7,7,HrRecruitmentProcess,over,183,2020-09-18 14:40:18.813368,End Point (HR Manager)->Odoo Application:[Http...,execute_kw,"['odoo01', '6', '123456789', 'hr.applicant', '...",hr.applicant_create,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
8,8,HrRecruitmentProcess,over,241,2020-09-18 14:48:34.772959,Odoo Application->db Server/Mail Server:[Pgsql...,NaN,NaN,NaN,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
9,9,HrRecruitmentProcess,over,243,2020-09-18 14:48:56.985180,Odoo Application->db Server/Mail Server:[Pgsql...,NaN,NaN,NaN,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0


In [26]:
pred = crf.predict(X_test_seq)

In [27]:
pred[0]

['GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity',
 'GenerateJobApplicationActivity']

In [28]:
def majority_voting(predictions):
    """
    Perform majority voting on each sublist of predictions.

    :param predictions: A list of lists where each sublist contains predictions.
    :return: A list of majority voted predictions for each sublist.
    """
    majority_voted_predictions = []
    for sublist in predictions:
        if not sublist:
            # If the sublist is empty, append None to the majority voted predictions
            majority_voted_predictions.append(None)
        else:
            # Count the occurrences of each prediction in the sublist
            counts = {}
            for prediction in sublist:
                if prediction in counts:
                    counts[prediction] += 1
                else:
                    counts[prediction] = 1
            
            # Find the prediction with the maximum count (mode)
            majority_voted_prediction = max(counts, key=counts.get)
            majority_voted_predictions.append(majority_voted_prediction)

    return majority_voted_predictions

maj_vote_pred = majority_voting(pred)
maj_vote_true = majority_voting(test_labels_seq_window)

In [29]:
from sklearn.metrics import classification_report

print(classification_report(maj_vote_true, maj_vote_pred))

                                 precision    recall  f1-score   support

               ContractProposal       1.00      1.00      1.00         1
 GenerateJobApplicationActivity       1.00      1.00      1.00        10
         PerformAnInterviewCall       1.00      1.00      1.00         5
      PerformAnInterviewMeeting       1.00      1.00      1.00         2
           ResumeReviewActivity       1.00      1.00      1.00        10
ScheduleAnInterviewActivityCall       1.00      1.00      1.00         5
     ScheduleAnInterviewMeeting       1.00      1.00      1.00         3

                       accuracy                           1.00        36
                      macro avg       1.00      1.00      1.00        36
                   weighted avg       1.00      1.00      1.00        36


# Case Id Matching

In [30]:
df_test

,Unnamed: 0,BusinessActivity,InstanceNumber,frame.number,synthetic_sniff_time,event_with_roles,request_method_call,file_data,selective_file_data,origin_method,origin_file_data,ActivityAction,stack_idx,activity_name,bp_id
0,0,HrRecruitmentProcess,over,17,2020-09-18 13:37:23.875912,End Point (HR Manager)->Odoo Application:[Http...,version,[],version,NaN,NaN,Activity Start,0,GenerateJobApplicationActivity,1224.0
1,1,HrRecruitmentProcess,over,49,2020-09-18 13:45:19.523472,Odoo Application->End Point (HR Manager):[Http...,server_version,"['server_version', '12.0-20190820', 'server_ve...",server_version,version,version,NoAction,0,GenerateJobApplicationActivity,1224.0
2,2,HrRecruitmentProcess,over,57,2020-09-18 13:45:48.836403,End Point (HR Manager)->Odoo Application:[Http...,authenticate,"['odoo01', 'dana.wireless@gmail.com', '1234567...",NaN,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
3,3,HrRecruitmentProcess,over,101,2020-09-18 14:30:06.242117,Odoo Application->db Server/Mail Server:[Pgsql...,NaN,NaN,NaN,NaN,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
4,4,HrRecruitmentProcess,over,110,2020-09-18 14:31:05.748067,Odoo Application->End Point (HR Manager):[Http...,IsNumber,['6'],IsNumber,authenticate,NaN,NoAction,0,GenerateJobApplicationActivity,1224.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,1308,HrRecruitmentProcess,over,28267,2020-09-29 19:59:51.450937,Odoo Application->db Server/Mail Server:[Pgsql...,NaN,NaN,NaN,NaN,NaN,NoAction,36,PerformAnInterviewMeeting,1231.0
1309,1309,HrRecruitmentProcess,over,28304,2020-09-29 20:02:07.104501,Odoo Application->End Point (Recruiting manage...,IsNumber,['1'],IsNumber,execute_kw,hr.applicant_write_stage_id_8,NoAction,36,PerformAnInterviewMeeting,1231.0
1310,1310,HrRecruitmentProcess,over,28312,2020-09-29 20:02:24.696260,End Point (Recruiting manager)->Odoo Applicati...,execute_kw,"['odoo01', '7', '123456789', 'hr.applicant', '...",hr.applicant_archive_applicant,NaN,NaN,NoAction,36,PerformAnInterviewMeeting,1231.0
1311,1311,HrRecruitmentProcess,over,28377,2020-09-29 20:11:57.543539,Odoo Application->db Server/Mail Server:[Pgsql...,NaN,NaN,NaN,NaN,NaN,NoAction,36,PerformAnInterviewMeeting,1231.0


In [31]:
df_features = pd.read_csv('../../data_v3/hr_il_tagged.csv')

In [32]:
attribute_cols = ["partner_name","message_id","parent_id","email","display_name","headers","references","partner_id","commercial_partner_id","date","mail_followers_id","date_last_stage_update","mail_message_id","mail_mail_id","mail_id","wizard_id","checksum","store_fname"]

df_features["partner_name"].value_counts()

partner_name
Daniel Duncan          1
Donna Fischer          1
Lala Funke             1
Ruben Jones            1
Julie James            1
Sara Williams          1
Rachel Mattlin         1
Kimberly Willingham    1
Seth Vescio            1
Frankie Hammes         1
Name: count, dtype: int64

In [40]:
df_features.sort_values(by=['BusinessActivity',"InstanceNumber","frame.number"])[["partner_name","BusinessActivity","InstanceNumber",]]

,partner_name,BusinessActivity,InstanceNumber
0,NaN,HrRecruitmentProcess,over
1,NaN,HrRecruitmentProcess,over
2,NaN,HrRecruitmentProcess,over
3,NaN,HrRecruitmentProcess,over
4,NaN,HrRecruitmentProcess,over
...,...,...,...
1293,NaN,HrRecruitmentProcess,over
1294,NaN,HrRecruitmentProcess,over
1295,NaN,HrRecruitmentProcess,over
1296,NaN,HrRecruitmentProcess,over


In [34]:
df_train["file_data"].value_counts()

file_data
                                                                                                                                                        22389
[]                                                                                                                                                       1169
[server_version, 12.0-20190820, server_version_info, 12, 0, 0, final, 0, server_serie, 12.0, protocol_version, 1]                                         932
[1]                                                                                                                                                       867
[6]                                                                                                                                                       846
                                                                                                                                                        ...  
[odoo01, 6, 123456789, hr.applicant, write

In [35]:
df_train[df_train["selective_file_data"] == "hr.applicant_create"]["file_data"].value_counts()

file_data
[odoo01, 6, 123456789, hr.applicant, create, name, Software Engineer, partner_name, Jennifer Hobbs, email_from, jennifer.hobbs@gmail.com, job_id, 2]         1
[odoo01, 6, 123456789, hr.applicant, create, name, Head of Research, partner_name, Leon Baker, email_from, leon.baker@gmail.com, job_id, 4]                  1
[odoo01, 6, 123456789, hr.applicant, create, name, Software Engineer, partner_name, Conrad Galloway, email_from, conrad.galloway@gmail.com, job_id, 2]       1
[odoo01, 6, 123456789, hr.applicant, create, name, Head of Research, partner_name, Joyce Shapero, email_from, joyce.shapero@gmail.com, job_id, 4]            1
[odoo01, 6, 123456789, hr.applicant, create, name, Sales Manager, partner_name, Debra Moretz, email_from, debra.moretz@gmail.com, job_id, 3]                 1
                                                                                                                                                            ..
[odoo01, 6, 123456789, hr.applicant,

In [36]:
df_train["file_data_str"] =df_train["file_data"].astype(str)

In [37]:
df_train["file_data_str"] 

0                                                       []
1        ['server_version', '12.0-20190820', 'server_ve...
2        ['odoo01', 'dana.wireless@gmail.com', '1234567...
3                                                         
4                                                    ['6']
                               ...                        
33068                                                     
33069                                                ['1']
33070    ['odoo01', '6', '123456789', 'hr.applicant', '...
33071                                                     
33072                                                   []
Name: file_data_str, Length: 33073, dtype: object

In [38]:
df_train

,BusinessActivity,InstanceNumber,frame.number,synthetic_sniff_time,event_with_roles,request_method_call,file_data,selective_file_data,origin_method,origin_file_data,SequenceNumber,file_data_str
0,GenerateJobApplicationActivity,1,26,2020-05-13 10:07:18.092710,End Point (HR Manager)->Odoo Application:[Http...,version,[],version,,,25,[]
1,GenerateJobApplicationActivity,1,127,2020-05-13 22:08:59.592427,Odoo Application->End Point (HR Manager):[Http...,server_version,"[server_version, 12.0-20190820, server_version...",server_version,version,version,25,"['server_version', '12.0-20190820', 'server_ve..."
2,GenerateJobApplicationActivity,1,135,2020-05-13 22:09:20.264494,End Point (HR Manager)->Odoo Application:[Http...,authenticate,"[odoo01, dana.wireless@gmail.com, 123456789]",,,,25,"['odoo01', 'dana.wireless@gmail.com', '1234567..."
3,GenerateJobApplicationActivity,1,192,2020-05-13 23:30:15.860005,Odoo Application->db Server/Mail Server:[Pgsql...,,,,,,25,
4,GenerateJobApplicationActivity,1,216,2020-05-13 23:55:12.809685,Odoo Application->End Point (HR Manager):[Http...,IsNumber,[6],IsNumber,authenticate,,25,['6']
...,...,...,...,...,...,...,...,...,...,...,...,...
33068,ResumeReviewActivity,250,729,2020-05-14 04:15:16.416366,Odoo Application->db Server/Mail Server:[Pgsql...,,,,,,728,
33069,ResumeReviewActivity,250,766,2020-05-14 04:17:47.741497,Odoo Application->End Point (HR Manager):[Http...,IsNumber,[1],IsNumber,execute_kw,hr.applicant_write_stage_id_2,728,['1']
33070,ResumeReviewActivity,250,774,2020-05-14 04:18:04.803203,End Point (HR Manager)->Odoo Application:[Http...,execute_kw,"[odoo01, 6, 123456789, hr.applicant, archive_a...",hr.applicant_archive_applicant,,,728,"['odoo01', '6', '123456789', 'hr.applicant', '..."
33071,ResumeReviewActivity,250,840,2020-05-14 04:28:47.207437,Odoo Application->db Server/Mail Server:[Pgsql...,,,,,,728,
